In [1]:
import numpy as np
import networkx as nx
import pandas as pd
from collections import defaultdict
import scipy.sparse
import scipy.sparse.linalg

In [2]:
dfEvents = pd.read_csv('data/events_elo.csv')

In [81]:
dfBatters = pd.read_csv('data/batters.csv')
dfPitchers = pd.read_csv('data/pitchers.csv')

In [32]:
dfBatters.head()

,player_id,teamID,nameFirst,nameLast,score
0,cabrm001,DET,Miguel,Cabrera,377.68
1,goldp001,ARI,Paul,Goldschmidt,349.07
2,troum001,LAA,Mike,Trout,347.80
3,canor001,NYA,Robinson,Cano,323.92
4,vottj001,CIN,Joey,Votto,317.49


In [3]:
dfEvents.head()

,game_dt,bat_id,pit_id,batter_score,pitcher_score
0,20130409,crisc001,wilsc004,0.0,1.0
1,20130409,younc004,wilsc004,0.0,1.0
2,20130409,lowrj001,wilsc004,1.0,0.0
3,20130409,cespy001,wilsc004,1.0,0.0
4,20130409,norrd001,wilsc004,1.0,0.0


In [36]:
def get_victories(df):
    batter_wins = defaultdict(lambda: defaultdict(int))
    pitcher_faces = defaultdict(set)
    
    for _, r in df.iterrows():
        b = r['bat_id']
        p = r['pit_id']
        pitcher_faces[p].add(b)
        if r['batter_score']: # I think it can be .5 but i dont feel like dealing with that
            batter_wins[b][p] += 1
    
    return batter_wins, pitcher_faces

# welcome to hell
def build_network(batter_wins, pitcher_faces):
    G = nx.MultiDiGraph()
    for b, d in batter_wins.items():
        for p, c in d.items():
            for b2 in pitcher_faces[p]:
                for _ in range(c):
                    if b != b2:
                        G.add_edge(b, b2)
                
    return G

def get_rankings(df):
    batter_wins, pitcher_faces = get_victories(df)
    G = build_network(batter_wins, pitcher_faces)
    A = nx.to_numpy_matrix(G)
    ranks = SpringRank(A)
    ids = G.nodes()
    ranks, ids = zip(*sorted(zip(ranks, ids), reverse=True))
    
    return list(ranks), list(ids)

In [37]:
ranks, ids = get_rankings(dfEvents)

In [84]:
names = []
for player_id in ids[3:]:
    dfPlayers = dfBatters.copy()
    dfPlayers.append(dfPitchers)
    r = dfPlayers.loc[dfPlayers['player_id'] == player_id]
    l = r['nameFirst'].tolist()
    if l:
#         print(player_id)
#         print(r)
        print(str(r['nameFirst']) + ' ' + str(r['nameLast']))
    break
    names.append(str(r['nameFirst']) + ' ' + str(r['nameLast']))

2    Mike
Name: nameFirst, dtype: object 2    Trout
Name: nameLast, dtype: object


In [53]:
print(names)

[]


In [16]:
def SpringRank(A,alpha=0.,l0=1.0,l1=1.0,solver='bicgstab',verbose=False):
    """
    Main routine to calculate SpringRank by solving linear system
    Default parameters are initialized as in the standard SpringRank model
    
    INPUT:

        A=network adjacency matrix (can be weighted)
        alpha: controls the impact of the regularization term
        l0: regularization spring's rest length
        l1: interaction springs' rest length
        solver: linear system solver. Options: 'spsolve'(direct, slower) or 'bicgstab' (iterative, faster)
        verbose: if True, then outputs some info about the numerical solvers

    OUTPUT:
        
        rank: N-dim array, indeces represent the nodes' indices used in ordering the matrix A

    """
    N = A.shape[0]
    k_in = np.sum(A, 0)
    k_out = np.sum(A, 1)
    one_vector = np.ones(N)

    C = A+A.T
    D1 = np.zeros(A.shape)
    D2 = np.zeros(A.shape)

    for i in range(A.shape[0]):
        D1[i,i] = k_out[i,0]+k_in[0,i]
        D2[i,i] = l1*(k_out[i,0]-k_in[0,i])

    if alpha != 0.: 
        if verbose:
            print('Using alpha!=0: matrix is invertible')

        B = one_vector*alpha*l0+np.dot(D2,one_vector)
        A = alpha*np.eye(N)+D1-C
        A = scipy.sparse.csr_matrix(np.matrix(A))

    else:    
        if verbose:
            print('alpha=0, using faster computation: fixing a rank degree of freedom')
        
        C += np.repeat(A[N-1,:][None],N,axis=0)+np.repeat(A[:,N-1].T[None], N, axis=0)
        D3 = np.zeros(A.shape)
        for i in range(A.shape[0]):
            D3[i,i] = l1*(k_out[N-1,0]-k_in[0,N-1])

        B = np.dot(D2,one_vector) + np.dot(D3,one_vector)
        A = scipy.sparse.csr_matrix(np.matrix(D1-C))
    
    if solver =='spsolve':
        if verbose:
            print('Using scipy.sparse.linalg.spsolve(A,B)')
        rank = scipy.sparse.linalg.spsolve(A,B)
    elif solver=='bicgstab': 
        if verbose:
            print('Using scipy.sparse.linalg.bicgstab(A,B)')
        rank = scipy.sparse.linalg.bicgstab(A,B)[0]
    else:
        print('Using scipy.sparse.linalg.bicgstab(A,B)')
        rank = scipy.sparse.linalg.bicgstab(A,B)[0]
    return np.transpose(rank)